In [53]:
import matplotlib.pyplot as pyp
import scipy.stats as st
import pandas as p
import numpy as n
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [54]:
def read_dataset(path, separ, en):
    return p.read_csv(path, sep = separ, encoding = en)

dataset = read_dataset("fullDataset.csv", ',', 'cp1252')
dataset.info()
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139483 entries, 0 to 139482
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  139483 non-null  int64  
 1   ID          139483 non-null  int64  
 2   Sex         139483 non-null  object 
 3   Age         139483 non-null  int64  
 4   Height      139483 non-null  int64  
 5   Weight      139483 non-null  int64  
 6   Team        139483 non-null  object 
 7   Year        139483 non-null  int64  
 8   Season      139483 non-null  object 
 9   Sport       139483 non-null  object 
 10  Medal       18968 non-null   object 
 11  Code        139483 non-null  object 
 12  GDP         139483 non-null  float64
 13  Population  139483 non-null  float64
dtypes: float64(2), int64(6), object(6)
memory usage: 11.7+ MB


,Unnamed: 0,ID,Sex,Age,Height,Weight,Team,Year,Season,Sport,Medal,Code,GDP,Population
0,0,1,M,24,180,80,China,1992,Summer,Basketball,NaN,CHN,4.269157e+11,1.164970e+09
1,1,2,M,23,170,60,China,2012,Summer,Judo,NaN,CHN,8.532230e+12,1.350695e+09
2,2,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
3,3,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
4,4,5,F,25,185,82,Netherlands,1992,Winter,Speed Skating,NaN,NLD,3.629629e+11,1.518417e+07


In [55]:
drop_list = ["Unnamed: 0", "Sex", "Age", "Height", "Weight", "Team", "Sport"]
dataset["GDP per capita"] = dataset["GDP"] / dataset["Population"]

for str in drop_list:
    dataset = dataset.drop(str, axis = 1)

dataset.info()
dataset.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139483 entries, 0 to 139482
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              139483 non-null  int64  
 1   Year            139483 non-null  int64  
 2   Season          139483 non-null  object 
 3   Medal           18968 non-null   object 
 4   Code            139483 non-null  object 
 5   GDP             139483 non-null  float64
 6   Population      139483 non-null  float64
 7   GDP per capita  139483 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 6.9+ MB


,ID,Year,Season,Medal,Code,GDP,Population,GDP per capita
0,1,1992,Summer,NaN,CHN,4.269157e+11,1.164970e+09,366.460692
1,2,2012,Summer,NaN,CHN,8.532230e+12,1.350695e+09,6316.918318
2,5,1988,Winter,NaN,NLD,2.619105e+11,1.476009e+07,17744.501384
3,5,1988,Winter,NaN,NLD,2.619105e+11,1.476009e+07,17744.501384
4,5,1992,Winter,NaN,NLD,3.629629e+11,1.518417e+07,23904.037415
5,5,1992,Winter,NaN,NLD,3.629629e+11,1.518417e+07,23904.037415
6,5,1994,Winter,NaN,NLD,3.791303e+11,1.538284e+07,24646.314310
7,5,1994,Winter,NaN,NLD,3.791303e+11,1.538284e+07,24646.314310
8,6,1992,Winter,NaN,USA,6.520327e+12,2.565140e+08,25418.990776
9,6,1992,Winter,NaN,USA,6.520327e+12,2.565140e+08,25418.990776


In [56]:
dataset = dataset.groupby(["Year", "Season", "Code"]).agg(GDP = ("GDP", "mean"),
                                                          Population = ("Population", "mean"),
                                                          Members = ("ID","nunique"),
                                                          GDP_p_c = ("GDP per capita", "mean"),
                                                          Medal = ("Medal", "count"))

dataset.info()

dataset.to_csv("check_r.csv", sep=',', encoding='utf-8')
dataset.head(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2258 entries, (1980, 'Summer', 'AFG') to (2016, 'Summer', 'ZWE')
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GDP         2258 non-null   float64
 1   Population  2258 non-null   float64
 2   Members     2258 non-null   int64  
 3   GDP_p_c     2258 non-null   float64
 4   Medal       2258 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 98.0+ KB


GDP   Population  Members       GDP_p_c  Medal
Year Season Code                                                         
1980 Summer AFG   3.641723e+09   13356511.0       11    272.655286      0
            AGO   5.934074e+09    8341289.0       11    711.409664      0
            AND   4.464161e+08      36067.0        2  12377.411646      0
            AUS   1.500323e+11   14692000.0      120  10211.837189     13
            AUT   8.205891e+10    7549433.0       83  10869.546494      5
            BEL   1.268293e+11    9859242.0       59  12864.002566      1
            BEN   1.405252e+09    3717165.0        7    378.043898      0
            BGR   1.983923e+10    8861535.0      264   2238.802958     89
            BRA   2.373935e+11  120694009.0      105   1966.903675      9
            BWA   1.060924e+09     897868.0        7   1181.603342      0

In [57]:
def kolm_smirn_check(dFrame, column):
    ks_statistic, p_value = st.kstest(dFrame[column], 'norm')
    if p_value > 0.05: return True
    return False

def pearson_check(dFrame, column):
    statistic, p_value = st.normaltest(dFrame[column])
    if p_value > 0.05: return True
    return False

print("#1 test - Kolmagorov-Smirnov test,\n#2 test - Pearson test\n")
for c in dataset.columns:
    print('{0:<15}'.format(c + ":"),f"#1 test - {kolm_smirn_check(dataset, c)}, #2 test - {pearson_check(dataset, c)};")

#1 test - Kolmagorov-Smirnov test,
#2 test - Pearson test

GDP:            #1 test - False, #2 test - False;
Population:     #1 test - False, #2 test - False;
Members:        #1 test - False, #2 test - False;
GDP_p_c:        #1 test - False, #2 test - False;
Medal:          #1 test - False, #2 test - False;


In [58]:
dataset.corr()

,GDP,Population,Members,GDP_p_c,Medal
GDP,1.000000,0.366452,0.547481,0.172987,0.612213
Population,0.366452,1.000000,0.244177,-0.072707,0.243602
Members,0.547481,0.244177,1.000000,0.149004,0.876835
GDP_p_c,0.172987,-0.072707,0.149004,1.000000,0.141599
Medal,0.612213,0.243602,0.876835,0.141599,1.000000


In [59]:
first = dataset.iloc[:, :4]
second = dataset['Medal']
f_train, f_test, s_train, s_test = train_test_split(first, second, test_size=0.3)

In [60]:
par = ["GDP", "Population", "Members", "GDP_p_c",
       ["GDP", "Population"],["GDP", "Members"],["GDP", "GDP_p_c"], ["Members", "Population"], ["Members", "GDP_p_c"], ["Population", "GDP_p_c"],
       ["GDP", "Population","Members"], ["GDP", "Population", "GDP_p_c"], ["GDP", "Members", "GDP_p_c"], ["Population", "Members", "GDP_p_c"],
       ["GDP", "Population", "Members", "GDP_p_c"]]

lin = []
pol = []
for i in range(len(par)):
    pol.append(make_pipeline(PolynomialFeatures(degree = 2), LinearRegression()))
    if i < 4: 
        lin.append(LinearRegression().fit(f_train[par[i]].to_numpy().reshape(-1,1), s_train))
        pol[i].fit(f_train[par[i]].to_numpy().reshape(-1,1), s_train)
    else:
        lin.append(LinearRegression().fit(f_train[par[i]], s_train))
        pol[i].fit(f_train[par[i]], s_train)

In [61]:
prediction = []

def predict_by(prediction, s):
    for i in range(len(par)):
        if i < 4:
            prediction.append(s[i].predict(f_test[par[i]].to_numpy().reshape(-1,1)))
        else:
            prediction.append(s[i].predict(f_test[par[i]]))
    return prediction

prediction = predict_by(prediction, lin)
prediction = predict_by(prediction, pol)

In [62]:
min_i = n.sum((n.array(prediction) - s_test.to_numpy())**2,axis = 1).argmin()
min= n.sum((n.array(prediction) - s_test.to_numpy())**2,axis = 1)
print(min)
print("Best model is:")
if (min_i < 15):
    print(f"linear model by {par[min_i]} params")
else:
    print(f"polynomial model by {par[min_i - 15]} params")

[336604.94078773 533230.85008924 146800.85483823 566541.08666532
 336265.28964063 126118.95688787 336129.60853575 146629.79462062
 146702.4390118  520548.12758521 126058.67798188 335633.56269416
 126231.26413107 146423.63700523 126095.13660488 314608.76135652
 473999.55348589  93668.35642021 550820.0847485  319983.9057867
 179972.6158186  320875.74068527 106144.91986064  92673.01159846
 327681.19615281 181416.87924794 319136.51687101 182189.22273132
  96673.414564   181244.19636432]
Best model is:
polynomial model by ['Members', 'GDP_p_c'] params


In [63]:
# from mpl_toolkits.mplot3d import Axes3D

# params = ['GDP', 'Population']
# params_values = []

# for i in range(len(params)):
#     values = n.linspace(f_test[params[i]].min(), f_test[params[i]].max()).reshape(-1, 1)
#     params_values.append(values)


# fig = pyp.figure(figsize=(8, 8))
# ax = fig.gca(projection='3d')

# X_3d = params_values[0]
# Y_3d = params_values[1]

# XX, YY = n.meshgrid(X_3d, Y_3d)

# Z = []
# for i in range(len(Y_3d)):
#     temp = []
#     for j in range(len(X_3d)):
#         temp.append(pol[2].predict(n.array([X_3d[j], Y_3d[i]]).T)[0])
#     Z.append(temp)
    
# Z = n.array(Z)

# ax.set_title('Поліноміальна регресія Medals ~ GDP, Population')
# ax.set_xlabel('GDP')
# ax.set_ylabel('Population')
# ax.set_zlabel('Medals')
# ax.plot_surface(
#     XX, YY,
#     n.array(Z),
#     color='green',
#     alpha=0.5
# )
# ax.scatter(f_test['GDP'], f_test['Population'], s_test)
# pyp.show()

In [64]:
arr = pol[2].predict(f_test[par[2]].to_numpy().reshape(-1,1))
sum = 0
count = 0
for i in arr:
    print(i)
    count += 1
    sum += i

print(f"avg {sum / count}")

0.6679286514425199
0.06700681018066892
0.4638903954192165
0.5654431514805324
0.4638903954192165
0.7713468953051791
0.3632703832585722
3.659944759589929
5.115611112208198
18.465411916011476
5.392403392483886
73.52696575536181
0.2635831149985995
0.4638903954192165
0.7713468953051791
0.5654431514805324
0.06700681018066892
0.3632703832585722
1.4114439803952374
0.3632703832585722
7.895775914785704
1.9705086752387553
8.051427817085974
1.1943473097625315
0.9809816147325119
-0.029882226377288948
0.2635831149985995
15.823527979962703
0.6679286514425199
4.30762012499725
25.163686968980294
0.8756978830685096
11.71175387456181
0.4638903954192165
1.087198090297186
0.4638903954192165
0.4638903954192165
78.58185816244803
1.8568302484687085
0.2635831149985995
0.06700681018066892
0.06700681018066892
0.3632703832585722
21.043985595926472
5.957180879843323
0.3632703832585722
0.4638903954192165
3.916216674050842
-0.029882226377288948
0.3632703832585722
0.2635831149985995
3.282532467153596
3.28253246715359

In [65]:
sum_d = 0
count_d = 0
for i in range(len(dataset)):
    count_d += 1
    sum_d += dataset.iloc[i, 4]

print(f"avg {sum_d / count_d}")

avg 14005.777315285626
